In [1]:
library("tidyverse")
library("data.table")
library("cowplot")

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘data.table’


The following objects are masked from ‘package:lubridate’:

    hour, isoweek, mday, minute, month, quarter, second, wday, week,
    yday, year


The following objects are masked from ‘package:dplyr’:

    between, first, last


The following object is masked from ‘package:purrr’:

    transpose



Attaching package: ‘cowplot’


The following object is masked from ‘package:lubridate’:

    stamp




In [2]:
invnorm <- function(x) {
  res <- rank(x)
  res <- qnorm(res/(length(res)+0.5))
  return(res)
}

In [3]:
extract_info <- function(the_var, sst, data, the_assay, the_state) {
    pval <- data[the_var, "Pr(>F)"]
    ss <- data["test_strain", "Sum Sq"]
    var_exp <- ss/sst
    ret <- data.table(assay = the_assay, var_name = the_var, hmm_state = the_state, pval = pval, var_exp = var_exp)
}

In [4]:
run_anova <- function(the_state, the_assay, data){
    fit <- lm(f_state_norm ~ date + time_numeric + tank_side + quadrant + test_strain, data = data[assay == the_assay & hmm_state == the_state])
    res <- aov(fit) |> summary()
    sst <- res[[1]][, "Sum Sq"] |> sum()
    ret <- lapply(
        rownames(res[[1]]) |> trimws(), 
        extract_info,
        sst = sst,
        data = res[[1]],
        the_assay = the_assay,
        the_state = the_state
    ) |> rbindlist()
    return(ret)
}

In [5]:
df <- fread("/nfs/research/birney/users/saul/nextflow/medaka_behaviour_pilot/hmm/time_step0.08_n_states15_hmm.csv.gz")
df[, mean_dist := log10(mean(distance)), by = hmm_state]
tmp <- df[, .(hmm_state, mean_dist)] |> distinct() |> as.data.table()
tmp[, hmm_state_recoded := rank(mean_dist)]
df <- merge(df, tmp, by = c("hmm_state", "mean_dist"))
head(df)

hmm_state,mean_dist,id,frame_n,time_s,distance,angle,hmm_state_recoded
<int>,<dbl>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
0,0.1005448,20190611_1331_icab_icab_R_no_q1_ref,757,25.23333,0.2083191,0.0000000,6
0,0.1005448,20190611_1331_icab_icab_R_no_q1_ref,761,25.36667,0.4425455,0.0000000,6
0,0.1005448,20190611_1331_icab_icab_R_no_q1_ref,1755,58.50000,1.2756671,-0.4597774,6
0,0.1005448,20190611_1331_icab_icab_R_no_q1_ref,1779,59.30000,1.3794776,-0.1900450,6
0,0.1005448,20190611_1331_icab_icab_R_no_q1_ref,1781,59.36667,1.3745458,-0.3463498,6
0,0.1005448,20190611_1331_icab_icab_R_no_q1_ref,1787,59.56667,1.4936201,-0.3414281,6


In [6]:
df_sum <- df[, .(id, hmm_state = hmm_state_recoded)] |>
    separate(
        id,
        into = c("date", "time_string", "ref_strain", "test_strain", "tank_side", "assay", "quadrant", "fish_type"),
        sep = "_",
        remove = FALSE
    ) |>
    as.data.table()

df_sum[, time_numeric_h := str_remove(time_string, "..$") |> as.numeric()]
df_sum[, time_numeric_m := str_remove(time_string, "^..") |> as.numeric()]
df_sum[, time_numeric := time_numeric_h + (time_numeric_m/60)]

head(df_sum)

id,date,time_string,ref_strain,test_strain,tank_side,assay,quadrant,fish_type,hmm_state,time_numeric_h,time_numeric_m,time_numeric
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
20190611_1331_icab_icab_R_no_q1_ref,20190611,1331,icab,icab,R,no,q1,ref,6,13,31,13.51667
20190611_1331_icab_icab_R_no_q1_ref,20190611,1331,icab,icab,R,no,q1,ref,6,13,31,13.51667
20190611_1331_icab_icab_R_no_q1_ref,20190611,1331,icab,icab,R,no,q1,ref,6,13,31,13.51667
20190611_1331_icab_icab_R_no_q1_ref,20190611,1331,icab,icab,R,no,q1,ref,6,13,31,13.51667
20190611_1331_icab_icab_R_no_q1_ref,20190611,1331,icab,icab,R,no,q1,ref,6,13,31,13.51667
20190611_1331_icab_icab_R_no_q1_ref,20190611,1331,icab,icab,R,no,q1,ref,6,13,31,13.51667


In [8]:
df_sge <- df_sum[
    fish_type == "ref" | test_strain == "icab", # I can use both fish in icab/icab pairs
    .(hmm_state, n_tot = .N), by = c("id", "date", "time_numeric", "test_strain", "tank_side", "assay", "quadrant", "fish_type")
][
    , .(f_state = .N/unique(n_tot)), by = c("hmm_state", "id", "date", "time_numeric", "test_strain", "tank_side", "assay", "quadrant", "fish_type")
]
head(df_sge)

hmm_state,id,date,time_numeric,test_strain,tank_side,assay,quadrant,fish_type,f_state
<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
6,20190611_1331_icab_icab_R_no_q1_ref,20190611,13.51667,icab,R,no,q1,ref,0.06634808
10,20190611_1331_icab_icab_R_no_q1_ref,20190611,13.51667,icab,R,no,q1,ref,0.02256057
11,20190611_1331_icab_icab_R_no_q1_ref,20190611,13.51667,icab,R,no,q1,ref,0.06868193
3,20190611_1331_icab_icab_R_no_q1_ref,20190611,13.51667,icab,R,no,q1,ref,0.06490331
9,20190611_1331_icab_icab_R_no_q1_ref,20190611,13.51667,icab,R,no,q1,ref,0.17470549
2,20190611_1331_icab_icab_R_no_q1_ref,20190611,13.51667,icab,R,no,q1,ref,0.06645921


In [9]:
df_sge[, .N, by = test_strain]

test_strain,N
<chr>,<int>
icab,4043
kaga,1382
hni,2234
hdr,1783
ho5,1678


In [10]:
df_sge[, .N, by = fish_type]

fish_type,N
<chr>,<int>
ref,9094
test,2026


In [11]:
df_sge[, .(res = sum(f_state) - 1 < .Machine$double.eps), by = id][, all(res)] |> stopifnot()

In [12]:
df_sge[, f_state_norm := invnorm(f_state), by = c("assay", "hmm_state")]

In [13]:
res <- list(
    of = lapply(1:15, run_anova, data = df_sge, the_assay = "of"),
    no = lapply(1:15, run_anova, data = df_sge, the_assay = "no")
) |>
    lapply(rbindlist) |>
    rbindlist()

In [14]:
head(res)

assay,var_name,hmm_state,pval,var_exp
<chr>,<chr>,<int>,<dbl>,<dbl>
of,date,1,4.823720e-01,0.08148103
of,time_numeric,1,8.988113e-01,0.08148103
of,tank_side,1,7.855785e-02,0.08148103
of,quadrant,1,6.099329e-02,0.08148103
of,test_strain,1,1.996251e-06,0.08148103
of,Residuals,1,NA,0.08148103


In [15]:
res <- res[var_name != "Residuals"]

In [16]:
res[, q := p.adjust(pval, method = "fdr")]
res[, sig := q < 0.05]

In [17]:
res[var_name == "test_strain", .(n = .N, min = min(q) |> signif(3), max = max(q) |> signif(3)), by = c("sig", "assay")]

sig,assay,n,min,max
<lgl>,<chr>,<int>,<dbl>,<dbl>
TRUE,of,2,1.58e-05,2.99e-05
FALSE,of,13,9.43e-02,8.91e-01
TRUE,no,3,1.90e-05,1.54e-02
FALSE,no,12,5.62e-02,4.26e-01


In [18]:
res[var_name == "test_strain", max(var_exp) |> signif(3)]

[1] 0.0864

In [19]:
res[var_name != "test_strain" & sig == TRUE, .(max = max(q), min = min(q)), by = c("assay", "var_name")]

assay,var_name,max,min
<chr>,<chr>,<dbl>,<dbl>
of,tank_side,0.01465638,3.689633e-03
of,date,0.03264000,5.079771e-08
of,time_numeric,0.03997067,2.313998e-04
no,date,0.01671990,4.406312e-08
no,time_numeric,0.03264000,4.912461e-03
no,tank_side,0.03264000,3.264000e-02
no,quadrant,0.04677343,4.677343e-02


In [20]:
res[var_name %in% c("quadrant", "date"), .(max = max(q) |> signif(3), min = min(q) |> signif(3)), by = c("assay")]

assay,max,min
<chr>,<dbl>,<dbl>
of,0.865,5.08e-08
no,0.881,4.41e-08


In [21]:
res <- res |> rstatix::add_significance(p.col = "q") |> as.data.table()
head(res)

assay,var_name,hmm_state,pval,var_exp,q,sig,q.signif
<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<lgl>,<chr>
of,date,1,4.823720e-01,0.081481028,5.930803e-01,FALSE,ns
of,time_numeric,1,8.988113e-01,0.081481028,9.048436e-01,FALSE,ns
of,tank_side,1,7.855785e-02,0.081481028,1.914089e-01,FALSE,ns
of,quadrant,1,6.099329e-02,0.081481028,1.663453e-01,FALSE,ns
of,test_strain,1,1.996251e-06,0.081481028,2.994376e-05,TRUE,****
of,date,2,1.128740e-01,0.003611797,2.351542e-01,FALSE,ns


In [22]:
pretty_df <- res[
    var_name == "test_strain",
    .(
        `Assay component` = ifelse(assay == "of", "Open field", "Novel object"),
        `HMM State` = hmm_state,
        `p-value (FDR-adjusted)` = signif(q, 3) |> format(scientific = TRUE),
        `Significance` = q.signif,
        `Variance explained` = signif(var_exp, 3)
    )
]

pretty_df

Assay component,HMM State,p-value (FDR-adjusted),Significance,Variance explained
<chr>,<int>,<chr>,<chr>,<dbl>
Open field,1,2.99e-05,****,0.08150
Open field,2,8.81e-01,ns,0.00361
Open field,3,8.91e-01,ns,0.00310
Open field,4,1.58e-05,****,0.08640
Open field,5,4.74e-01,ns,0.01020
Open field,6,4.45e-01,ns,0.01080
Open field,7,6.82e-01,ns,0.00696
Open field,8,2.40e-01,ns,0.01690
Open field,9,6.74e-01,ns,0.00709


In [23]:
fwrite(pretty_df, "tableS2.csv")